In [2]:
import pandas as pd

In [4]:
import numpy as np
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors

In [6]:
b=pd.read_csv('books/books.csv')

In [7]:
b.head()

,book_id,goodreads_book_id,best_book_id,work_id,books_count,isbn,isbn13,authors,original_publication_year,original_title,...,ratings_count,work_ratings_count,work_text_reviews_count,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5,image_url,small_image_url
0,1,2767052,2767052,2792775,272,439023483,9.780439e+12,Suzanne Collins,2008.0,The Hunger Games,...,4780653,4942365,155254,66715,127936,560092,1481305,2706317,https://images.gr-assets.com/books/1447303603m...,https://images.gr-assets.com/books/1447303603s...
1,2,3,3,4640799,491,439554934,9.780440e+12,"J.K. Rowling, Mary GrandPré",1997.0,Harry Potter and the Philosopher's Stone,...,4602479,4800065,75867,75504,101676,455024,1156318,3011543,https://images.gr-assets.com/books/1474154022m...,https://images.gr-assets.com/books/1474154022s...
2,3,41865,41865,3212258,226,316015849,9.780316e+12,Stephenie Meyer,2005.0,Twilight,...,3866839,3916824,95009,456191,436802,793319,875073,1355439,https://images.gr-assets.com/books/1361039443m...,https://images.gr-assets.com/books/1361039443s...
3,4,2657,2657,3275794,487,61120081,9.780061e+12,Harper Lee,1960.0,To Kill a Mockingbird,...,3198671,3340896,72586,60427,117415,446835,1001952,1714267,https://images.gr-assets.com/books/1361975680m...,https://images.gr-assets.com/books/1361975680s...
4,5,4671,4671,245494,1356,743273567,9.780743e+12,F. Scott Fitzgerald,1925.0,The Great Gatsby,...,2683664,2773745,51992,86236,197621,606158,936012,947718,https://images.gr-assets.com/books/1490528560m...,https://images.gr-assets.com/books/1490528560s...


In [15]:
print(b.shape)
print(b.columns)

(10000, 23)
Index(['book_id', 'goodreads_book_id', 'best_book_id', 'work_id',
       'books_count', 'isbn', 'isbn13', 'authors', 'original_publication_year',
       'original_title', 'title', 'language_code', 'average_rating',
       'ratings_count', 'work_ratings_count', 'work_text_reviews_count',
       'ratings_1', 'ratings_2', 'ratings_3', 'ratings_4', 'ratings_5',
       'image_url', 'small_image_url'],
      dtype='object')


In [8]:
r=pd.read_csv('ratings/ratings.csv')

In [9]:
r.head()

,user_id,book_id,rating
0,1,258,5
1,2,4081,4
2,2,260,5
3,2,9296,5
4,2,2318,3


In [32]:
r.columns,r.shape 

(Index(['user_id', 'book_id', 'rating'], dtype='object'), (5976479, 3))

In [10]:
bt=pd.read_csv('book_tags/book_tags.csv')
bt.head()

,goodreads_book_id,tag_id,count
0,1,30574,167697
1,1,11305,37174
2,1,11557,34173
3,1,8717,12986
4,1,33114,12716


In [11]:
t=pd.read_csv('tags/tags.csv')
t.head()

,tag_id,tag_name
0,0,-
1,1,--1-
2,2,--10-
3,3,--12-
4,4,--122-


In [12]:
to_read=pd.read_csv('to_read/to_read.csv')
to_read.head()

,user_id,book_id
0,9,8
1,15,398
2,15,275
3,37,7173
4,34,380


In [19]:
books_df =b[['book_id','title']]
rating_df = r[['user_id', 'book_id', 'rating']]

In [21]:
df = pd.merge(rating_df,books_df,on='book_id')
df.head()

,user_id,book_id,rating,title
0,1,258,5,The Shadow of the Wind (The Cemetery of Forgot...
1,11,258,3,The Shadow of the Wind (The Cemetery of Forgot...
2,143,258,4,The Shadow of the Wind (The Cemetery of Forgot...
3,242,258,5,The Shadow of the Wind (The Cemetery of Forgot...
4,325,258,4,The Shadow of the Wind (The Cemetery of Forgot...


In [22]:
combine_book_rating = df.dropna(axis = 0, subset = ['title'])
book_ratingCount = (combine_book_rating.
     groupby(by = ['title'])['rating'].
     count().
     reset_index().
     rename(columns = {'rating': 'totalRatingCount'})
     [['title', 'totalRatingCount']]
    )
book_ratingCount.head()

,title,totalRatingCount
0,"Angels (Walsh Family, #3)",263
1,"""حكايات فرغلي المستكاوي ""حكايتى مع كفر السحلاوية",113
2,#GIRLBOSS,195
3,'Salem's Lot,4433
4,"'Tis (Frank McCourt, #2)",703


In [27]:
rating_with_totalRatingCount = combine_book_rating.merge(book_ratingCount, left_on = 'title', right_on = 'title', how = 'left')
rating_with_totalRatingCount.head()

,user_id,book_id,rating,title,totalRatingCount
0,1,258,5,The Shadow of the Wind (The Cemetery of Forgot...,3531
1,11,258,3,The Shadow of the Wind (The Cemetery of Forgot...,3531
2,143,258,4,The Shadow of the Wind (The Cemetery of Forgot...,3531
3,242,258,5,The Shadow of the Wind (The Cemetery of Forgot...,3531
4,325,258,4,The Shadow of the Wind (The Cemetery of Forgot...,3531


In [28]:
# pd.set_option('display.float_format', lambda x: '%.3f' % x)
print(book_ratingCount['totalRatingCount'].describe())

count     9964.000000
mean       599.807206
std       1269.924727
min          8.000000
25%        155.000000
50%        249.000000
75%        508.250000
max      22806.000000
Name: totalRatingCount, dtype: float64


In [33]:
popularity_threshold = 500
rating_popular_book= rating_with_totalRatingCount.query('totalRatingCount >= @popularity_threshold')
rating_popular_book.head()

,user_id,book_id,rating,title,totalRatingCount
0,1,258,5,The Shadow of the Wind (The Cemetery of Forgot...,3531
1,11,258,3,The Shadow of the Wind (The Cemetery of Forgot...,3531
2,143,258,4,The Shadow of the Wind (The Cemetery of Forgot...,3531
3,242,258,5,The Shadow of the Wind (The Cemetery of Forgot...,3531
4,325,258,4,The Shadow of the Wind (The Cemetery of Forgot...,3531


In [34]:
rating_popular_book.shape

(4339157, 5)

In [35]:
book_features_df=rating_popular_book.pivot_table(index='title',columns='user_id',values='rating').fillna(0)
book_features_df.head()

user_id,1,2,3,4,5,6,7,8,9,10,...,53415,53416,53417,53418,53419,53420,53421,53422,53423,53424
title,,,,,,,,,,,,,,,,,,,,,
'Salem's Lot,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0
"'Tis (Frank McCourt, #2)",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11/22/63,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0
"13 Little Blue Envelopes (Little Blue Envelope, #1)",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1776,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0


In [36]:
book_features_df_matrix = csr_matrix(book_features_df.values)

model_knn = NearestNeighbors(metric = 'cosine', algorithm = 'brute')
model_knn.fit(book_features_df_matrix)

NearestNeighbors(algorithm='brute', leaf_size=30, metric='cosine',
                 metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                 radius=1.0)

In [37]:
query_index = np.random.choice(book_features_df.shape[0])
print(query_index)
distances, indices = model_knn.kneighbors(book_features_df.iloc[query_index,:].values.reshape(1, -1), n_neighbors = 6)

1899


In [38]:
for i in range(0, len(distances.flatten())):
    if i == 0:
        print('Recommendations for {0}:\n'.format(book_features_df.index[query_index]))
    else:
        print('{0}: {1}, with distance of {2}:'.format(i, book_features_df.index[indices.flatten()[i]], distances.flatten()[i]))

Recommendations for The Iron King (The Iron Fey, #1):

1: The Iron Daughter, with distance of 0.2845535356342883:
2: The Iron Queen (The Iron Fey, #3), with distance of 0.32850135548796944:
3: The Iron Knight (The Iron Fey, #4), with distance of 0.43624081991441366:
4: Hush, Hush (Hush, Hush, #1), with distance of 0.710712039032565:
5: Unearthly (Unearthly, #1), with distance of 0.7263190930730294:
